__TF-IDF and N-Grams__

The goal of this project was to predict the sentiment of an IMDB movie review using a binary classification system. The dataset was part of the Bag of Words Meets Bag of Popcorn Competition.

Model Accuracy: 0.89532

Bag of Words & TF-IDF
A Bag of Words (BoW) model is a simple algorithm used in Natural Language Processing. It simply counts the number of times a word appears in a document.

__Feature Generation using TF-IDF__

In Term Frequency(TF), you just count the number of words occurred in each document. The main issue with this Term Frequency is that it will give more weight to longer documents. Term frequency is basically the output of the BoW model.

IDF(Inverse Document Frequency) measures the amount of information a given word provides across the document. IDF is the logarithmically scaled inverse ratio of the number of documents that contain the word and the total number of documents.


TF-IDF(Term Frequency-Inverse Document Frequency) normalizes the document term matrix. It is the product of TF and IDF. Word with high tf-idf in a document, it is most of the times occurred in given documents and must be absent in the other documents. So the words must be a signature word.



In the project, I used two separate TF-IDF vectorizers and merged them into a single bag of words.

-  The first vectorizer (word_vectorizer) analyzed complete words.
-  The second vectorizer (char_vectorizer) analyzed the frequency of character n-grams. An n-gram is a continous sequence of n items from a document. Using Trigrams (N-gram size = 3) yielded a high predictive score.

Lastly, we used a Logistic Regression to predict the sentiment attached to each review. The hyperparameters of the model were tuned using a validation dataset prior to training the model

# Loading Required Libraries and Reading the Data into Python
First, we need to load the required libraries and read the data into Python.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.sparse import hstack
from time import time

In [3]:
train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t")
test = pd.read_csv("testData.tsv", header=0, delimiter="\t")

train_text = train['review']
test_text = test['review']
y = train['sentiment']

all_text = pd.concat([train_text, test_text])

__TF-IDF Vectorizers__

First, we convert the reviews into a Bag of Words using the TF-IDF vectorizer for words and for character trigrams.

In [4]:
word_vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', sublinear_tf=True, strip_accents='unicode', 
                                  stop_words='english', ngram_range=(1, 1), max_features=10000)
word_vectorizer.fit(train_text)

train_word_features = word_vectorizer.transform(train_text)

In [6]:
char_vectorizer = TfidfVectorizer(analyzer='char', sublinear_tf=True, strip_accents='unicode', 
                                  stop_words='english', ngram_range=(1, 3), max_features=50000)
char_vectorizer.fit(train_text)

train_char_features = char_vectorizer.transform(train_text)

In [7]:
train_features = hstack([train_word_features, train_char_features])

__Hyperparameter Tuning of Logistic Regression__

Since there are multiple hyperparameters to tune in the XGBoost model, we will use the GridSearchCV function of Sklearn to determine the optimal hyperparameter values. Next, I used the train_test_split function to generate a validation set and find the best parameters.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train_features, y,test_size=0.3 ,random_state=1234)

lr_model = LogisticRegression(random_state=1234)
param_dict = {'C': [0.001, 0.01, 0.1, 1, 10],
             'solver': ['sag', 'lbfgs', 'saga']}

start = time()
grid_search = GridSearchCV(lr_model, param_dict)
grid_search.fit(X_train, y_train)
print("GridSearch took %.2f seconds to complete." % (time()-start))
display(grid_search.best_params_)
print("Cross-Validated Score of the Best Estimator: %.3f" % grid_search.best_score_)

C:\Users\khata\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\khata\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


GridSearch took 323.94 seconds to complete.


{'C': 1, 'solver': 'saga'}

Cross-Validated Score of the Best Estimator: 0.888


Let's see how well our model does on the validation dataset and where any misclassifications occur.

We have several metrics available for classification accuracy, including a confusion matrix and a classification report.

In [9]:
lr=LogisticRegression(C=1, solver ='saga')
lr.fit(X_train, y_train)
lr_preds=lr.predict(X_test)

print(confusion_matrix(y_test, lr_preds))
print(classification_report(y_test, lr_preds))
print("Accuracy Score: %.3f" % accuracy_score(y_test, lr_preds))

[[3367  398]
 [ 368 3367]]
              precision    recall  f1-score   support

           0       0.90      0.89      0.90      3765
           1       0.89      0.90      0.90      3735

    accuracy                           0.90      7500
   macro avg       0.90      0.90      0.90      7500
weighted avg       0.90      0.90      0.90      7500

Accuracy Score: 0.898


The number of false positives (FP = 366) is similar to the number of false negatives (FN = 399), suggesting that our model is not biased towards either specificity nor sensitivity.

# Model Building and Evaluation (TF-IDF)

Model Building and Evaluation (TF-IDF)
Let's build the Text Classification Model using TF-IDF.

First, import the LogisticRegression module using the tuned hyperparameters.
Then, fit your model on a train set using fit() and perform prediction on the test set using predict()
Format predictions for submission to Kaggle Competition.

In [11]:
word_vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', sublinear_tf=True, strip_accents='unicode', 
                                  stop_words='english', ngram_range=(1, 1), max_features=10000)
word_vectorizer.fit(all_text)

train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [12]:
char_vectorizer = TfidfVectorizer(analyzer='char', sublinear_tf=True, strip_accents='unicode', 
                                  stop_words='english', ngram_range=(1, 3), max_features=50000)
char_vectorizer.fit(all_text)

train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [13]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [14]:
lr=LogisticRegression(C=1,solver='saga')
lr.fit(train_features,y)
final_preds=lr.predict(test_features)

Making submission for Kaggle.

In [15]:
test['sentiment'] = final_preds
test = test[['id', 'sentiment']]
test.to_csv('Submissionn.csv',index=False)